In [1]:
%cd ..

/media/Z/NDT/PytorchLightning/4. NLP


In [2]:
from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)

In [3]:
from model import NERModelModule
from configs import *

In [4]:
%%capture
model_name_or_path=BASE_MODEL_NAME
model_ner = NERModelModule(
    model_name_or_path = model_name_or_path, 
    num_labels = len(TAGS),
    tags_list = TAGS
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [5]:
BEST_CHECKPOINT = "/media/Z/NDT/PytorchLightning/4. NLP/artifact/weight/jd_extract_1673564916.0833418/epoch=23--val_overall_f1=0.83.ckpt"

In [6]:
%%capture
model_ner.load_from_checkpoint(BEST_CHECKPOINT)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [7]:
filepath = "/media/Z/NDT/PytorchLightning/4. NLP/artifact/onnx/model_v1.onnx"

In [8]:
import torch

In [9]:
input_sample = {
            'input_ids': torch.randint(3, 2000, (16, 128)).type(torch.LongTensor) ,
            'attention_mask':  torch.ones(16, 128),
        }

In [10]:
dynamic_axes={
    "input_ids": {0: 'batch', 1: 'sequence'},
    "attention_mask": {0: 'batch', 1: 'sequence'},
    "last_hidden_state": {0: "batch"}
    }
input_names=["input_ids","attention_mask"]
output_names=['last_hidden_state']

In [11]:
from torch.onnx import export as onnx_export

In [12]:
onnx_export(
        model_ner.model,
        (input_sample,),
        filepath,
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axes,
        do_constant_folding=True,
        opset_version=13,
        )